In [1]:
# ロジスティック回帰による分類予測
# - bank-fully のy列（ていき預金を申し込んだか否か）について分類予測する
# - 特徴量には簡単のため、数値の列と文字列の*default*列のみ使用する
# - 数値の列は標準化を行う
# - 文字列の列はインデックス化する
# - 制度評価として混同行例とAUCを計算する

In [2]:
import numpy as np 
import pandas as pd 

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("logistic_regression").getOrCreate()

In [4]:
filename = "./data/bank/bank-full.csv"
data = spark.read.csv(filename, header=True, inferSchema=True, sep=";")

In [6]:
data.dtypes

[('age', 'int'),
 ('job', 'string'),
 ('marital', 'string'),
 ('education', 'string'),
 ('default', 'string'),
 ('balance', 'int'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('contact', 'string'),
 ('day', 'int'),
 ('month', 'string'),
 ('duration', 'int'),
 ('campaign', 'int'),
 ('pdays', 'int'),
 ('previous', 'int'),
 ('poutcome', 'string'),
 ('y', 'string')]

In [10]:
# 目的変数の作成 yes: 1, no:0
from pyspark.sql.functions import lit, when, col
data1 = data.withColumn("y1", when(col("y") == "yes", lit(1.0)).otherwise(lit(0.0)))

In [11]:
data1.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y| y1|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+---+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|0.0|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|0.0|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|0.0|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|0.0|
| 33|     unknown|  single|  unknown|     no|      1|  

In [12]:
# String の処理(default)
from pyspark.ml.feature import StringIndexer
default_index = StringIndexer(inputCol="default", outputCol="default_index")

In [13]:
# assemble
from pyspark.ml.feature import VectorAssembler
assemble = VectorAssembler(inputCols=["age","balance","duration","campaign","previous", "default_index"]
                          ,outputCol="features")

In [15]:
# 標準化
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features",outputCol="scaled_faetures")

In [16]:
# ロジスティック回帰
from pyspark.ml.classification import LogisticRegression
logistic_regression = LogisticRegression(featuresCol="scaled_faetures", labelCol="y1")

In [18]:
# パイプラインの登録
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[default_index, assemble, scaler, logistic_regression])

In [19]:
df = data1.select("age","balance","duration","campaign","previous", "default","y","y1")

In [21]:
# 訓練データとテストデータを分ける
train_df, test_df = df.randomSplit([0.7, 0.3],seed=1234)

In [22]:
# 訓練データによるモデリング
fit_model = pipeline.fit(train_df)

In [23]:
# モデリング結果の確認
fit_model.stages[3].coefficients

DenseVector([0.0461, 0.1101, 0.9227, -0.4444, 0.2843, -0.0603])

In [24]:
fit_model.stages[3].intercept

-3.2359670441544646

In [26]:
# 訓練データを使って推論
pred_train = fit_model.transform(train_df)

In [27]:
pred_train.show()

+---+-------+--------+--------+--------+-------+---+---+-------------+--------------------+--------------------+--------------------+--------------------+----------+
|age|balance|duration|campaign|previous|default|  y| y1|default_index|            features|     scaled_faetures|       rawPrediction|         probability|prediction|
+---+-------+--------+--------+--------+-------+---+---+-------------+--------------------+--------------------+--------------------+--------------------+----------+
| 18|     35|     104|       2|       0|     no| no|0.0|          0.0|[18.0,35.0,104.0,...|[1.69579509967235...|[3.06350348098030...|[0.95536194323606...|       0.0|
| 18|    108|      92|       1|       1|     no|yes|1.0|          0.0|[18.0,108.0,92.0,...|[1.69579509967235...|[2.81052152711537...|[0.94324174651528...|       0.0|
| 18|    108|     169|       1|       0|     no|yes|1.0|          0.0|[18.0,108.0,169.0...|[1.69579509967235...|[2.68263210486711...|[0.93599399206992...|       0.0|
| 18

In [31]:
pred_train.select("rawPrediction", "probability").show(truncate=False)

+----------------------------------------+-----------------------------------------+
|rawPrediction                           |probability                              |
+----------------------------------------+-----------------------------------------+
|[3.063503480980308,-3.063503480980308]  |[0.9553619432360639,0.04463805676393606] |
|[2.8105215271153705,-2.8105215271153705]|[0.9432417465152872,0.05675825348471275] |
|[2.6826321048671122,-2.6826321048671122]|[0.9359939920699278,0.06400600793007216] |
|[1.7481311839187115,-1.7481311839187115]|[0.8517169345743544,0.14828306542564562] |
|[2.105961860368693,-2.105961860368693]  |[0.8914812910541506,0.10851870894584936] |
|[1.741314289879969,-1.741314289879969]  |[0.8508539277745946,0.14914607222540543] |
|[2.3641117736081894,-2.3641117736081894]|[0.9140493896360997,0.08595061036390028] |
|[2.308834902187733,-2.308834902187733]  |[0.90960610335261,0.09039389664738995]   |
|[3.071697801995959,-3.071697801995959]  |[0.9557100931299582,0.0

In [33]:
# z は線形予測値
z = np.array([3.063503480980308,-3.063503480980308])
q = 1 / (1 + np.exp(-z))
print(q)

[0.95536194 0.04463806]


In [35]:
# 精度評価（訓練データ） AUCの算出
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [36]:
evaluator = BinaryClassificationEvaluator(labelCol = "y1")
AUC = evaluator.evaluate(pred_train)
print(AUC)

0.8331547859811365


In [37]:
# テストデータ
test_df.show()

+---+-------+--------+--------+--------+-------+---+---+
|age|balance|duration|campaign|previous|default|  y| y1|
+---+-------+--------+--------+--------+-------+---+---+
| 18|      3|     130|       2|       0|     no|yes|1.0|
| 18|      5|     143|       2|       0|     no| no|0.0|
| 18|    108|     167|       1|       0|     no|yes|1.0|
| 19|    103|      96|       2|       2|     no| no|0.0|
| 19|    108|     168|       1|       2|     no|yes|1.0|
| 19|    108|     273|       2|       1|     no|yes|1.0|
| 19|    134|     271|       2|       0|     no|yes|1.0|
| 19|    179|      62|       3|       0|     no| no|0.0|
| 19|    291|     291|       5|       0|     no| no|0.0|
| 19|    329|     169|       1|       2|     no|yes|1.0|
| 19|    329|     252|       2|       0|     no|yes|1.0|
| 19|    526|     122|       3|       0|     no| no|0.0|
| 19|   1803|     124|       1|       1|     no| no|0.0|
| 20|   -322|      73|       4|       0|     no| no|0.0|
| 20|     29|      85|       2|

In [38]:
test_df.count()

13562

In [39]:
# テストデータの推論
pred_test = fit_model.transform(test_df)

In [40]:
pred_test.show()

+---+-------+--------+--------+--------+-------+---+---+-------------+--------------------+--------------------+--------------------+--------------------+----------+
|age|balance|duration|campaign|previous|default|  y| y1|default_index|            features|     scaled_faetures|       rawPrediction|         probability|prediction|
+---+-------+--------+--------+--------+-------+---+---+-------------+--------------------+--------------------+--------------------+--------------------+----------+
| 18|      3|     130|       2|       0|     no|yes|1.0|          0.0|[18.0,3.0,130.0,2...|[1.69579509967235...|[2.97025888993604...|[0.95121229283086...|       0.0|
| 18|      5|     143|       2|       0|     no| no|0.0|          0.0|[18.0,5.0,143.0,2...|[1.69579509967235...|[2.92298874908348...|[0.94897122314169...|       0.0|
| 18|    108|     167|       1|       0|     no|yes|1.0|          0.0|[18.0,108.0,167.0...|[1.69579509967235...|[2.68989335994989...|[0.93642763358003...|       0.0|
| 19

In [41]:
AUC = evaluator.evaluate(pred_test)
print(AUC)

0.8321723113642512


In [43]:
# 混同行列
from sklearn.metrics import confusion_matrix
y_true = pred_test.select("y1")
y_true = y_true.toPandas()
y_true

,y1
0,1.0
1,0.0
2,1.0
3,0.0
4,1.0
...,...
13557,0.0
13558,1.0
13559,1.0
13560,0.0


In [45]:
y_pred = pred_test.select("prediction")
y_pred = y_pred.toPandas()
y_pred

,prediction
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
13557,0.0
13558,1.0
13559,1.0
13560,0.0


In [46]:
class_name = [0, 1]
cnf_matrix = confusion_matrix(y_true, y_pred, labels=class_name)
cnf_matrix

array([[11749,   214],
       [ 1304,   295]])

In [48]:
tn, fp, fn, tp = cnf_matrix.flatten()
print(tn, fp, fn, tp)

11749 214 1304 295


In [49]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [50]:
print("accuracy:{}".format(accuracy_score(y_true, y_pred)))

accuracy:0.8880696062527651


In [51]:
print("precision:{}".format(precision_score(y_true, y_pred)))
print("recall:{}".format(recall_score(y_true, y_pred)))
print("f1:{}".format(f1_score(y_true, y_pred)))

precision:0.5795677799607073
recall:0.18449030644152595
f1:0.27988614800759015
